# Balloon Navigation Control Demo - Complete Version

This notebook demonstrates tree search planning and PID control for balloon navigation, comparing open-loop vs closed-loop control strategies.

## Learning Objectives
- Understand how tree search generates navigation plans
- Compare open-loop vs closed-loop control performance
- Explore PID controller tuning and its effects
- Visualize trajectory tracking and error accumulation

## Navigation Task
**Route**: Ithaca, NY → Montreal, Canada (415 km)
**Challenge**: Navigate through realistic wind patterns with altitude control

## 🔧 Modification Instructions
**Cell 3**: Replace synthetic plan with your tree search output
**Cell 4**: Enhance wind field model with real data

In [ ]:
# Install and import dependencies
!pip install numpy matplotlib scipy

import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Dict, Any
import math

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print("✅ Dependencies installed and imported!")

In [ ]:
# Configuration
START_STATE = {
    'lat': 42.400,  # Ithaca, NY
    'lon': -76.500,
    'alt': 15.0     # km
}

TARGET_STATE = {
    'lat': 45.500,  # Montreal, Canada
    'lon': -73.600,
    'alt': 18.0     # km
}

# Simulation Parameters
DT = 3600  # Time step: 1 hour
SIMULATION_HOURS = 24
N_STEPS = SIMULATION_HOURS

# PID Control Parameters
PID_GAINS = {
    'conservative': {'Kp': 0.1, 'Ki': 0.01, 'Kd': 0.05},
    'balanced': {'Kp': 0.3, 'Ki': 0.05, 'Kd': 0.1},
    'aggressive': {'Kp': 0.5, 'Ki': 0.1, 'Kd': 0.15},
    'overshoot': {'Kp': 0.8, 'Ki': 0.02, 'Kd': 0.05},
    'slow': {'Kp': 0.1, 'Ki': 0.2, 'Kd': 0.02}
}

# Control scaling factors
LAT_CONTROL_SCALE = 0.001  # Degrees per control unit
LON_CONTROL_SCALE = 0.001
ALT_CONTROL_SCALE = 0.1    # km per control unit

print("Configuration loaded successfully!")
print(f"Start: {START_STATE['lat']:.3f}°N, {START_STATE['lon']:.3f}°E at {START_STATE['alt']} km")
print(f"Target: {TARGET_STATE['lat']:.3f}°N, {TARGET_STATE['lon']:.3f}°E at {TARGET_STATE['alt']} km")

In [ ]:
# Utility functions (including haversine_distance)
def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """Calculate distance between two points on Earth's surface."""
    R = 6371000  # Earth's radius in meters

    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = math.sin(dlat/2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))

    return R * c

print("✅ Utility functions loaded!")

## 🔄 CELL 4: TREE SEARCH PLAN INPUT

**MODIFY THIS CELL** to load your tree search plan from a separate notebook.

Replace the synthetic plan generation with your actual tree search output.

In [ ]:
# ============================================================================
# OPTIONAL: Replace with your tree search plan
# ============================================================================

# Option 1: Load from file (if you saved tree search results)
# import json
# with open('tree_search_plan.json', 'r') as f:
#     tree_search_plan = json.load(f)

# Option 2: Direct input (paste your tree search results here)
# tree_search_plan = [
#     {'lat': 42.400, 'lon': -76.500, 'alt': 15.0, 'time_step': 0},
#     {'lat': 42.450, 'lon': -76.450, 'alt': 15.3, 'time_step': 1},
#     # ... add your complete plan here
# ]

# Option 3: Synthetic plan (current fallback)
def generate_tree_search_plan():
    """Generate a synthetic tree search plan with realistic progression."""
    plan = []

    # Calculate total distance and direction
    total_distance = haversine_distance(
        START_STATE['lat'], START_STATE['lon'],
        TARGET_STATE['lat'], TARGET_STATE['lon']
    )

    # Direction vector
    lat_diff = TARGET_STATE['lat'] - START_STATE['lat']
    lon_diff = TARGET_STATE['lon'] - START_STATE['lon']

    # Generate planned states with realistic progression
    n_planned_states = min(21, N_STEPS + 1)

    # Initialize
    plan.append({
            'lat': START_STATE['lat'],
            'lon': START_STATE['lon'],
            'alt': START_STATE['alt'],
            'control_lat': 0,
            'control_lon': 0,
            'control_alt': 0,
            'time_step': 0
        })

    for i in range(1, n_planned_states):
        # Progress fraction (non-linear for more realistic planning)
        progress = (i / (n_planned_states - 1)) ** 1.5  # Slightly faster at start

        # Add some planned deviations for altitude changes and wind compensation
        alt_progress = progress * (TARGET_STATE['alt'] - START_STATE['alt'])

        # Planned state
        planned_lat = START_STATE['lat'] + lat_diff * progress
        planned_lon = START_STATE['lon'] + lon_diff * progress
        planned_alt = START_STATE['alt'] + alt_progress

        # Add small planned deviations (simulating tree search finding optimal path)
        deviation_factor = 0.1 * math.sin(i * 0.5)  # Small sinusoidal deviations
        planned_lat += deviation_factor * 0.01
        planned_lon += deviation_factor * 0.01

        plan.append({
            'lat': planned_lat,
            'lon': planned_lon,
            'alt': planned_alt,
            'control_lat': planned_lat - plan[-1]['lat'],
            'control_lon': planned_lon - plan[-1]['lon'],
            'control_alt': planned_alt - plan[-1]['alt'],
            'time_step': i
        })

    return plan

tree_search_plan = generate_tree_search_plan()

print(f"📋 Loaded {len(tree_search_plan)} planned states")
# print(f"📏 Plan distance: {haversine_distance(tree_search_plan[0]['lat'], tree_search_plan[0]['lon'],
#                                           tree_search_plan[-1]['lat'], tree_search_plan[-1]['lon']):.0f} m")
# print("⚠️  Remember to replace with your actual tree search results!")

## 🌪️ CELL 5: ENHANCED WIND FIELD

**OPTIONAL MODIFICATION**: Enhance wind field with real data or more sophisticated models.

In [ ]:
# ============================================================================
#   ️ OPTIONAL: Enhance wind field with real data
# ============================================================================

# Option 1: Use ERA5 wind data (requires additional setup)
# !pip install cdsapi xarray
# import xarray as xr
# def get_era5_wind(lat, lon, alt, time_step):
#     # Load ERA5 data and interpolate
#     # This would require API access to Copernicus Climate Data Store
#     pass

# Option 2: Enhanced synthetic wind with more realistic patterns
def get_enhanced_wind(lat, lon, alt, time_step):
    """Enhanced wind model with jet stream and seasonal effects."""
    # Jet stream effect (stronger at higher altitudes and latitudes)
    jet_stream = 20 * math.exp(-((alt - 12) / 3)**2) * math.exp(-((lat - 45) / 10)**2)

    # Seasonal variation
    seasonal_factor = 1 + 0.3 * math.sin(time_step * 0.1 + lat * 0.1)

    # Coriolis effect (stronger at higher latitudes)
    coriolis_factor = 1 + abs(lat - 40) * 0.02

    # Base wind with all effects
    wind_speed = (8 + jet_stream) * seasonal_factor * coriolis_factor

    # Wind direction with realistic patterns
    base_dir = 45 + 30 * math.sin(time_step * 0.08)
    lat_effect = 15 * math.sin(lat * 0.15)

    wind_direction = base_dir + lat_effect

    wind_u = wind_speed * math.cos(math.radians(wind_direction))
    wind_v = wind_speed * math.sin(math.radians(wind_direction))

    return wind_u, wind_v

# Use enhanced wind model
get_wind_at_location = get_enhanced_wind

print("🌪️ Enhanced wind field model loaded!")
print("💡 You can replace this with real ERA5 or other wind data")

In [ ]:
# Additional utility functions
def update_state(lat: float, lon: float, alt: float, control_lat: float,
                control_lon: float, control_alt: float, time_step: int) -> Tuple[float, float, float]:
    """Update balloon state based on wind and control inputs."""
    # Get wind at current location
    wind_u, wind_v = get_wind_at_location(lat, lon, alt, time_step)

    # Convert wind to lat/lon changes (approximate)
    # 1 m/s ≈ 0.00001 degrees per second at mid-latitudes
    wind_lat_change = wind_v * 0.00001 * DT
    wind_lon_change = wind_u * 0.00001 * DT / math.cos(math.radians(lat))

    # Apply control inputs
    control_lat_change = control_lat * LAT_CONTROL_SCALE
    control_lon_change = control_lon * LON_CONTROL_SCALE
    control_alt_change = control_alt * ALT_CONTROL_SCALE

    # Update state
    new_lat = lat + wind_lat_change + control_lat_change
    new_lon = lon + wind_lon_change + control_lon_change
    new_alt = alt + control_alt_change

    # Keep altitude within reasonable bounds
    new_alt = max(10.0, min(25.0, new_alt))

    return new_lat, new_lon, new_alt

print("✅ Additional utility functions loaded!")

In [ ]:
# PID Controller and simulation
class PIDController:
    def __init__(self, Kp: float, Ki: float, Kd: float):
        self.Kp = Kp
        self.Ki = Ki
        self.Kd = Kd
        self.prev_error = 0
        self.integral = 0

    def compute(self, error: float, dt: float) -> float:
        """Compute PID control output."""
        self.integral += error * dt
        derivative = (error - self.prev_error) / dt if dt > 0 else 0

        output = self.Kp * error + self.Ki * self.integral + self.Kd * derivative

        self.prev_error = error
        return output

    def reset(self):
        """Reset controller state."""
        self.prev_error = 0
        self.integral = 0

def run_simulation_with_control(control_type: str, pid_gains: Dict[str, float] = None) -> Dict[str, Any]:
    """Run simulation with specified control strategy."""
    # Initialize state
    current_lat = START_STATE['lat']
    current_lon = START_STATE['lon']
    current_alt = START_STATE['alt']

    # Initialize PID controllers if needed
    if control_type == 'pid':
        lat_pid = PIDController(pid_gains['Kp'], pid_gains['Ki'], pid_gains['Kd'])
        lon_pid = PIDController(pid_gains['Kp'], pid_gains['Ki'], pid_gains['Kd'])
        alt_pid = PIDController(pid_gains['Kp'], pid_gains['Ki'], pid_gains['Kd'])

    # Storage for trajectory
    trajectory = []
    errors = []

    for step in range(N_STEPS):
        # Get planned state for this step
        plan_idx = min(step, len(tree_search_plan) - 1)
        planned_state = tree_search_plan[plan_idx]

        # Calculate errors
        lat_error = planned_state['lat'] - current_lat
        lon_error = planned_state['lon'] - current_lon
        alt_error = planned_state['alt'] - current_alt

        # Calculate control inputs
        if control_type == 'open_loop':
            # Open-loop: use planned trajectory directly
            control_lat = planned_state['control_lat']
            control_lon = planned_state['control_lat']
            control_alt = planned_state['control_lat']
        elif control_type == 'pid':
            # PID control
            control_lat = lat_pid.compute(lat_error, DT)
            control_lon = lon_pid.compute(lon_error, DT)
            control_alt = alt_pid.compute(alt_error, DT)

        # Update state
        new_lat, new_lon, new_alt = update_state(
            current_lat, current_lon, current_alt,
            control_lat, control_lon, control_alt, step
        )

        # Store trajectory
        trajectory.append({
            'lat': current_lat,
            'lon': current_lon,
            'alt': current_alt,
            'time_step': step
        })

        # Calculate tracking error
        tracking_error = haversine_distance(
            current_lat, current_lon,
            planned_state['lat'], planned_state['lon']
        )
        errors.append(tracking_error)

        # Update current state
        current_lat, current_lon, current_alt = new_lat, new_lon, new_alt

    # Add final state
    trajectory.append({
        'lat': current_lat,
        'lon': current_lon,
        'alt': current_alt,
        'time_step': N_STEPS
    })

    return {
        'trajectory': trajectory,
        'errors': errors,
        'final_error': errors[-1] if errors else 0,
        'avg_error': np.mean(errors) if errors else 0
    }

print("✅ PID Controller and simulation functions loaded!")

In [ ]:
# Run main comparison
print("=== Balloon Navigation Control Demo ===")
print(f"Start State:  Lat={START_STATE['lat']:.3f}, Lon={START_STATE['lon']:.3f}, Alt={START_STATE['alt']} km")
print(f"Target State: Lat={TARGET_STATE['lat']:.3f}, Lon={TARGET_STATE['lon']:.3f}, Alt={TARGET_STATE['alt']} km")

total_distance = haversine_distance(
    START_STATE['lat'], START_STATE['lon'],
    TARGET_STATE['lat'], TARGET_STATE['lon']
)
print(f"Distance: {total_distance:.0f} m\n")

print("=== Getting Tree Search Plan ===")
print(f"Generated {len(tree_search_plan)} planned states\n")

print("=== Open-Loop vs PID Control Comparison ===")

# Run open-loop simulation
open_loop_result = run_simulation_with_control('open_loop')

# Run PID simulation with balanced gains
pid_result = run_simulation_with_control('pid', PID_GAINS['balanced'])

print("\n=== Performance Comparison ===")
print(f"Open-Loop Control:")
print(f"  Final Error: {open_loop_result['final_error']:.0f} m")
print(f"  Avg Tracking Error: {open_loop_result['avg_error']:.0f} m")
print(f"PID Control:")
print(f"  Final Error: {pid_result['final_error']:.0f} m")
print(f"  Avg Tracking Error: {pid_result['avg_error']:.0f} m")

# Calculate improvements
final_improvement = ((open_loop_result['final_error'] - pid_result['final_error']) / open_loop_result['final_error']) * 100
avg_improvement = ((open_loop_result['avg_error'] - pid_result['avg_error']) / open_loop_result['avg_error']) * 100

print(f"Improvement:")
print(f"  Final Error Reduction: {final_improvement:.1f}%")
print(f"  Tracking Error Reduction: {avg_improvement:.1f}%")

In [ ]:
# PID gain comparison
print("\n=== PID Gain Comparison for Students ===")

pid_results = {}

for gain_name, gains in PID_GAINS.items():
    print(f"\nTesting {gain_name.title()}...")
    result = run_simulation_with_control('pid', gains)
    pid_results[gain_name] = result

    print(f"  Final Error: {result['final_error']:.0f} m")
    print(f"  Avg Tracking Error: {result['avg_error']:.0f} m")

# Find best performing configuration
best_gain = min(pid_results.keys(), key=lambda k: pid_results[k]['final_error'])
print(f"\n🏆 Best Performance: {best_gain.title()} gains")
print(f"   Final Error: {pid_results[best_gain]['final_error']:.0f} m")

In [ ]:
# Comprehensive visualization
def create_comprehensive_visualization():
    """Create comprehensive visualization of all results."""
    fig = plt.figure(figsize=(20, 16))

    # 1. Main trajectory comparison
    ax1 = plt.subplot(2, 3, 1)

    # Plot planned trajectory
    plan_lats = [state['lat'] for state in tree_search_plan]
    plan_lons = [state['lon'] for state in tree_search_plan]
    ax1.plot(plan_lons, plan_lats, 'b--', linewidth=3, label='Tree Search Plan', alpha=0.8)

    # Plot open-loop trajectory
    ol_lats = [state['lat'] for state in open_loop_result['trajectory']]
    ol_lons = [state['lon'] for state in open_loop_result['trajectory']]
    ax1.plot(ol_lons, ol_lats, 'r-', linewidth=2, label='Open-Loop Control', alpha=0.7)

    # Plot PID trajectory
    pid_lats = [state['lat'] for state in pid_result['trajectory']]
    pid_lons = [state['lon'] for state in pid_result['trajectory']]
    ax1.plot(pid_lons, pid_lats, 'g-', linewidth=2, label='PID Control', alpha=0.7)

    # Mark start and target
    ax1.plot(START_STATE['lon'], START_STATE['lat'], 'ko', markersize=10, label='Start (Ithaca)')
    ax1.plot(TARGET_STATE['lon'], TARGET_STATE['lat'], 'k*', markersize=15, label='Target (Montreal)')

    ax1.set_xlabel('Longitude')
    ax1.set_ylabel('Latitude')
    ax1.set_title('Trajectory Comparison\nIthaca, NY → Montreal, Canada')
    ax1.legend()
    ax1.grid(True, alpha=0.3)

    # 2. Error comparison over time
    ax2 = plt.subplot(2, 3, 2)
    time_hours = np.arange(len(open_loop_result['errors']))
    ax2.plot(time_hours, open_loop_result['errors'], 'r-', linewidth=2, label='Open-Loop')
    ax2.plot(time_hours, pid_result['errors'], 'g-', linewidth=2, label='PID Control')
    ax2.set_xlabel('Time (hours)')
    ax2.set_ylabel('Tracking Error (m)')
    ax2.set_title('Tracking Error Over Time')
    ax2.legend()
    ax2.grid(True, alpha=0.3)

    # 3. Altitude profiles
    ax3 = plt.subplot(2, 3, 3)
    plan_alts = [state['alt'] for state in tree_search_plan]
    plan_times = [state['time_step'] for state in tree_search_plan]
    ax3.plot(plan_times, plan_alts, 'b--', linewidth=3, label='Planned Altitude', alpha=0.8)

    ol_alts = [state['alt'] for state in open_loop_result['trajectory']]
    ol_times = [state['time_step'] for state in open_loop_result['trajectory']]
    ax3.plot(ol_times, ol_alts, 'r-', linewidth=2, label='Open-Loop Altitude', alpha=0.7)

    pid_alts = [state['alt'] for state in pid_result['trajectory']]
    pid_times = [state['time_step'] for state in pid_result['trajectory']]
    ax3.plot(pid_times, pid_alts, 'g-', linewidth=2, label='PID Altitude', alpha=0.7)

    ax3.set_xlabel('Time (hours)')
    ax3.set_ylabel('Altitude (km)')
    ax3.set_title('Altitude Control Comparison')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

    # 4. PID gain comparison
    ax4 = plt.subplot(2, 3, 4)
    gain_names = list(pid_results.keys())
    final_errors = [pid_results[name]['final_error'] for name in gain_names]

    colors = ['lightblue', 'lightgreen', 'orange', 'pink', 'lightcoral']
    bars = ax4.bar(gain_names, final_errors, color=colors, alpha=0.7)

    # Highlight best performance
    best_idx = gain_names.index(best_gain)
    bars[best_idx].set_color('gold')
    bars[best_idx].set_alpha(0.9)

    ax4.set_ylabel('Final Error (m)')
    ax4.set_title('PID Gain Performance Comparison')
    ax4.tick_params(axis='x', rotation=45)

    # Add value labels on bars
    for bar, error in zip(bars, final_errors):
        height = bar.get_height()
        ax4.text(bar.get_x() + bar.get_width()/2., height + 5000,
                f'{error:.0f}m', ha='center', va='bottom', fontsize=9)

    # 5. PID gain trajectories
    ax5 = plt.subplot(2, 3, 5)

    # Plot planned trajectory as reference
    ax5.plot(plan_lons, plan_lats, 'k--', linewidth=2, alpha=0.5, label='Planned')

    # Plot different PID gain trajectories
    line_styles = ['-', '--', '-.', ':', '-']
    markers = ['o', 's', '^', 'D', 'v']

    for i, (gain_name, result) in enumerate(pid_results.items()):
        traj_lats = [state['lat'] for state in result['trajectory']]
        traj_lons = [state['lon'] for state in result['trajectory']]

        # Plot every 4th point to avoid clutter
        ax5.plot(traj_lons[::4], traj_lats[::4],
                line_styles[i], marker=markers[i], markersize=4,
                label=f'{gain_name.title()}', alpha=0.8)

    ax5.plot(START_STATE['lon'], START_STATE['lat'], 'ko', markersize=10)
    ax5.plot(TARGET_STATE['lon'], TARGET_STATE['lat'], 'k*', markersize=15)

    ax5.set_xlabel('Longitude')
    ax5.set_ylabel('Latitude')
    ax5.set_title('PID Gain Trajectory Comparison')
    ax5.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax5.grid(True, alpha=0.3)

    # 6. Wind field visualization
    ax6 = plt.subplot(2, 3, 6)
    lats = np.linspace(40, 50, 10)
    lons = np.linspace(-80, -70, 10)
    LAT, LON = np.meshgrid(lats, lons)
    U, V = np.zeros_like(LAT), np.zeros_like(LON)

    for i in range(len(lats)):
        for j in range(len(lons)):
            u, v = get_wind_at_location(LAT[i,j], LON[i,j], 15, 12)
            U[i,j], V[i,j] = u, v

    ax6.quiver(LON, LAT, U, V, alpha=0.6)
    ax6.set_xlabel('Longitude')
    ax6.set_ylabel('Latitude')
    ax6.set_title('Wind Field at t=12h, Alt=15km')
    ax6.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Print educational summary
    print("\n" + "="*60)
    print("EDUCATIONAL SUMMARY")
    print("="*60)
    print(f"• Navigation Challenge: {total_distance/1000:.1f} km from Ithaca to Montreal")
    print(f"• Open-Loop Performance: {open_loop_result['final_error']/1000:.1f} km final error")
    print(f"• PID Control Improvement: {final_improvement:.1f}% error reduction")
    print(f"• Best PID Gains: {best_gain.title()} (Kp={PID_GAINS[best_gain]['Kp']}, ")
    print(f"  Ki={PID_GAINS[best_gain]['Ki']}, Kd={PID_GAINS[best_gain]['Kd']})")
    print(f"• Key Insight: Closed-loop control reduces error accumulation")
    print(f"• Learning: PID tuning significantly affects long-distance navigation")
    print("="*60)

# Create the visualization
create_comprehensive_visualization()

## 📝 Summary and Key Takeaways

### What We Demonstrated:
1. **Tree Search Planning**: Optimal path generation (replace with your results)
2. **Wind Field Effects**: Realistic atmospheric modeling with jet streams
3. **Control Comparison**: Open-loop vs closed-loop performance
4. **PID Tuning**: Impact of different gain configurations
5. **Error Accumulation**: How errors compound over long distances

### Key Modifications Made:
- **Cell 4**: Clear instructions for tree search plan input
- **Cell 5**: Enhanced wind field with jet stream effects
- **Wind Visualization**: Shows atmospheric patterns

### To Use Your Tree Search Results:
1. Run your tree search notebook separately
2. Copy the output plan to Cell 4
3. Optionally enhance wind field in Cell 5
4. Run all cells to see results

### Applications:
- High-altitude balloon navigation
- Autonomous vehicle path following
- Drone navigation in windy conditions
- Satellite orbit control